# 🍏 Health & Fitness Agent with Bing Grounding 🍎

Welcome to our **Health & Fitness Agent with Bing Grounding** tutorial! In this notebook, we'll demonstrate how to:

1. **Initialize** a project using Azure AI Foundry.
2. **Create an Agent** with the **BingGroundingTool** for web search.
3. **Ask real-world questions** about health and fitness.
4. **Retrieve and display** answers, including Bing query URLs and disclaimers.

### ⚠️ Important Model Support Note ⚠️
> Bing grounding is currently only supported in certain Azure OpenAI models (e.g. `gpt-4o-0513`).
> 
> Make sure you specify a supported model and set the `"x-ms-enable-preview": "true"` header.

## Prerequisites
- Grounding with Bing connection in Azure AI Foundry, which has to be provisioned from Azure portal.
See ["Setup Bing Grounding"](https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/bing-grounding?tabs=python&pivots=overview#setup) in the documentation for full details.


- A `cred.json` file in the parent directory containing:
  ```bash
  PROJECT_ENDPOINT=<your-endpoint>
  MODEL_DEPLOYMENT_NAME=<supported-model>
  BING_CONNECTION_NAME=<the-name-of-your-bing-connection>
  ```

## Let's Explore Grounding with Bing!
We'll integrate **Grounding with Bing** search results into our agent so it can gather extra context from the web. We'll store and display the Bing search query link for transparency. 🎉



## 1. Initial Setup
We'll load environment variables from `cred.json` and initialize our **AIProjectClient** to manage agents.

In [ ]:
# Copy and paste this entire code block into a single Jupyter cell

# =============================================================================
# BING SEARCH AGENT SETUP
# =============================================================================
# Before running this code, you need to:
# 1. Create a Bing Search connection in your Azure AI project
# 2. Get the connection ID from the Azure portal
# 3. Add "BING_CONNECTION_NAME": "your-connection-name" to your cred.json file
# 
# HOW TO FIND YOUR CONNECTION ID:
# 1. Go to Azure Portal (portal.azure.com)
# 2. Navigate to your Azure AI project
# 3. Go to "Connections" in the left menu
# 4. Find your Bing Search connection
# 5. Click on it to see details
# 6. Copy the "Connection ID" (not the name)
# 
# Example cred.json structure:
# {
#   "PROJECT_ENDPOINT": "your-connection-string",
#   "MODEL_DEPLOYMENT_NAME": "gpt-4o-mini",
#   "BING_CONNECTION_NAME": "your-actual-connection-id-from-portal"
# }
# =============================================================================

# Import required libraries
import os  # For environment variables and path operations
import time  # For adding delays if needed
from pathlib import Path  # For cross-platform path handling
import json

# Import Azure and utility libraries
from azure.identity import DefaultAzureCredential  # For Azure authentication
from azure.ai.projects import AIProjectClient  # Main client for AI Projects

# Try importing models from azure-ai-agents for Bing search functionality
try:
    from azure.ai.agents.models import (
        BingGroundingTool,
        MessageTextContent,
        MessageRole
    )
    print("✅ Successfully imported Bing search models from azure-ai-agents")
    BING_MODELS_AVAILABLE = True
except ImportError as e:
    print(f"❌ Could not import Bing search models from azure-ai-agents: {e}")
    BING_MODELS_AVAILABLE = False

# -----------------------------
# Load credentials function
# -----------------------------
def find_cred_json(start_path):
    current = Path(start_path)
    while current != current.parent:
        cred_file = current / 'cred.json'
        if cred_file.exists():
            return str(cred_file)
        current = current.parent
    return None

def load_credentials():
    """Load credentials from cred.json file"""
    cred_file = find_cred_json(os.getcwd())
    if not cred_file:
        raise FileNotFoundError("cred.json not found in current or parent directories.")
    
    with open(cred_file, "r") as f:
        cfg = json.load(f)
    
    return {
        "PROJECT_ENDPOINT": cfg.get("PROJECT_ENDPOINT"),
        "MODEL_DEPLOYMENT_NAME": cfg.get("MODEL_DEPLOYMENT_NAME", "gpt-4o"),
        "BING_CONNECTION_NAME": cfg.get("BING_CONNECTION_NAME")  # Use connection name instead of ID
    }

# -----------------------------
# Initialize AIProjectClient
# -----------------------------
def initialize_ai_project_client():
    """Initialize AIProjectClient with proper endpoint extraction"""
    try:
        config = load_credentials()

        # Initialize AIProjectClient
        client = AIProjectClient(
            endpoint=config["PROJECT_ENDPOINT"],
            credential=DefaultAzureCredential()
        )
        
        print("✅ Successfully initialized AIProjectClient")
        return client, config
        
    except Exception as e:
        print(f"❌ Error initializing AIProjectClient: {e}")
        return None, None

# -----------------------------
# List available connections function
# -----------------------------
def list_available_connections(client):
    """List all available connections in the project"""
    try:
        print("🔍 Checking available connections...")
        
        # List all connections
        connections = client.connections.list()
        connection_list = list(connections)
        
        print("📋 Available connections:")
        bing_connections = []
        
        for connection in connection_list:
            print(f"  - Name: {connection['name']}")
            print(f"    Type: {connection['type']}")
            print(f"    Target: {connection['target']}")
            print(f"    ID: {connection['id']}")  # The connection ID is the full path
            print()
            
            # Check if this is a Bing connection
            if (connection['type'] == 'ApiKey' and 
                ('bing' in connection['target'].lower() or 
                 'bing' in connection['name'].lower())):
                bing_connections.append(connection['id'])  # Use the full ID path
        
        if bing_connections:
            print(f"🎯 Found Bing connections: {bing_connections}")
            print(f"💡 Use this connection name in your cred.json: 'aibinggrounding'")
        else:
            print("⚠️ No Bing connections found. You may need to create one.")
            
        return connection_list
            
    except Exception as e:
        print(f"❌ Error listing connections: {e}")
        return []

# -----------------------------
# Find connection ID by name function
# -----------------------------
def find_connection_id_by_name(client, connection_name):
    """Find the full connection ID by matching the connection name"""
    try:
        connections = client.connections.list()
        for connection in connections:
            if connection['name'] == connection_name:
                print(f"✅ Found connection: {connection_name}")
                print(f"🔗 Using connection ID: {connection['id']}")
                return connection['id']
        
        print(f"❌ Connection '{connection_name}' not found")
        return None
    except Exception as e:
        print(f"❌ Error finding connection: {e}")
        return None


## 2. Create Bing-Grounded Agent 🌐
We'll fetch our Bing connection from AI Foundry and use `BingGroundingTool` to let our agent search the web. Then we'll create a new agent with disclaimers about not being a doctor, etc.

Make sure your `MODEL_DEPLOYMENT_NAME` is set to a Bing-supported model (for example, `gpt-4o-0513`) and that you add the header `{"x-ms-enable-preview": "true"}`.

In [ ]:
# -----------------------------
# Create Bing search agent function
# -----------------------------
def create_bing_search_agent(client, model_name, bing_connection_name=None):
    """Create a Bing search agent for real-time information"""
    try:
        print("🔍 Creating Bing search agent...")
        
        if BING_MODELS_AVAILABLE:
            # Check if we have a Bing connection name
            if not bing_connection_name:
                print("⚠️ No Bing connection name provided. You need to:")
                print("1. Create a Bing Search connection in your Azure AI project")
                print("2. Set the connection name in your cred.json file as 'BING_CONNECTION_NAME'")
                print("\n🔍 Let's check what connections are available...")
                list_available_connections(client)
                return None
            
            # Find the connection ID by name
            bing_connection_id = find_connection_id_by_name(client, bing_connection_name)
            if not bing_connection_id:
                return None
            
            # Create a BingGroundingTool for real-time web search
            bing_tool = BingGroundingTool(connection_id=bing_connection_id)
            
            # Create an AI agent that will use Bing search for real-time information
            agent = client.agents.create_agent(
                model=model_name,
                name="bing-search-agent",
                instructions="""
                    You are a helpful assistant with access to real-time web search through Bing.
                    You:
                    1. Use Bing search to find current, accurate information
                    2. Provide up-to-date news, weather, stock prices, and other real-time data
                    3. Always cite your sources when using search results
                    4. Be helpful and informative in your responses
                    5. If you can't find information, let the user know
                """,
                tools=bing_tool.definitions,
                tool_resources=bing_tool.resources
            )
            print(f"🎉 Created Bing search agent, ID: {agent.id}")
            return agent
        else:
            print("⚠️ Bing search agent creation not available - missing models")
            return None
        
    except Exception as e:
        print(f"❌ Error creating Bing search agent: {e}")
        return None



## 3. Starting Threads & Asking Questions 💬
We'll create conversation threads for each user query, letting the agent search with Bing to find relevant info. We will store all `(thread, run)` pairs in a list so we can review them in the next step.

In [ ]:
# -----------------------------
# Create search thread function
# -----------------------------
def create_search_thread(client):
    """Create a new conversation thread for Bing search"""
    try:
        thread = client.agents.threads.create()
        print(f"📝 Created new search thread, ID: {thread.id}")
        return thread
    except Exception as e:
        print(f"❌ Error creating search thread: {e}")
        return None

# -----------------------------
# Ask Bing search question function
# -----------------------------
def ask_bing_search_question(client, thread_id, agent_id, user_question):
    """Ask a question that will trigger Bing search"""
    try:
        # Add the user's question as a message to the thread
        message = client.agents.messages.create(
            thread_id=thread_id,
            role="user",
            content=user_question
        )
        print(f"🔍 Searching: '{user_question}'")

        # Create and process a run - this will trigger Bing search if needed
        run = client.agents.runs.create_and_process(
            thread_id=thread_id,
            agent_id=agent_id
        )
        print(f"🤖 Run finished with status: {run.status}")
        
        if run.last_error:
            print(f"Error details: {run.last_error}")
        return run
    except Exception as e:
        print(f"❌ Error searching question: {e}")
        return None



## 4. Viewing Bing-Grounded Answers & Query URLs
We’ll retrieve each thread's messages, printing both the user queries and the agent's responses. We'll also fetch the run steps to display the **Bing Search Query URL** (so you can comply with the requirement to show where the data came from). You can replace `api.bing.microsoft.com` with `www.bing.com` to form a user-friendly link.

Because `RunStep` objects do **not** have `.details`, we look instead for `'request_url'` in `step["parameters"]`. If found, it's presumably the Bing step.

In [ ]:
# -----------------------------
# Display thread messages function
# -----------------------------
def display_thread_messages(client, thread_id):
    """Display conversation history and search results"""
    try:
        # Retrieve all messages in this conversation thread
        messages = client.agents.messages.list(thread_id=thread_id)
        
        # Convert ItemPaged to list for easier handling
        messages_list = list(messages)

        # Display the conversation history in reverse chronological order (newest first)
        print("\n🗣️ Conversation so far:")
        for m in reversed(messages_list):
            if m.content:
                last_content = m.content[-1]
                if hasattr(last_content, "text"):
                    print(f"{m.role.upper()}: {last_content.text.value}\n")

        # Check for search citations and grounding information
        print("\n📎 Checking for search citations...")
        citation_found = False
        for m in messages_list:
            if hasattr(m, 'file_citation_annotations') and m.file_citation_annotations:
                for c in m.file_citation_annotations:
                    print(f"- Citation snippet: '{c.text}' from file ID: {c.file_citation['file_id']}")
                    citation_found = True
        
        if not citation_found:
            print("No citations found in the conversation.")

    except Exception as e:
        print(f"❌ Error displaying messages: {e}")
        import traceback
        traceback.print_exc()

# -----------------------------
# Test Bing search agent function
# -----------------------------
def test_bing_search_agent(client, agent):
    """Test the Bing search agent with sample queries"""
    if not agent:
        print("⚠️ Bing search agent testing not available - no agent created")
        return
    
    try:
        # Create a new conversation thread
        search_thread = create_search_thread(client)

        if search_thread:
            # Define some test questions that would benefit from real-time search
            queries = [
                "What's the current weather in New York?",
                "What are the latest news about artificial intelligence?",
                "What's the current stock price of Microsoft?",
                "What are the top trending topics on social media today?"
            ]

            # Process each query one at a time
            for q in queries:
                ask_bing_search_question(client, search_thread.id, agent.id, q)
                time.sleep(2)  # Add a small delay between queries
            
            # Display the conversation history and search results
            display_thread_messages(client, search_thread.id)
        
    except Exception as e:
        print(f"❌ Error testing Bing search agent: {e}")



## 5. Let's Ask Question
You can optionally delete the agent once you're done. In production, you might keep it around for repeated usage.


In [ ]:
# -----------------------------
# Main execution
# -----------------------------
print("🚀 Starting Bing Search Agent Setup")

# Initialize variables for cleanup
agent = None

try:
    # Initialize AIProjectClient
    client, config = initialize_ai_project_client()
    if not client:
        print("❌ Could not initialize AIProjectClient. Exiting.")
        exit()

    # Check if we can access agents
    try:
        agents = client.agents.list_agents()
        print("✅ Agents functionality is available")
        print(f"📋 Found {len(list(agents))} existing agents")
    except Exception as e:
        print(f"❌ Agents functionality check failed: {e}")

    # List available connections to help with setup
    print("\n🔍 Listing available connections...")
    connections = list_available_connections(client)

    # Check if Bing models are available
    if BING_MODELS_AVAILABLE:
        print("✅ BingGroundingTool and related models are available")
        
        # Check if we have a Bing connection name
        bing_connection_name = config.get("BING_CONNECTION_NAME")
        if not bing_connection_name:
            print("⚠️ No BING_CONNECTION_NAME found in cred.json")
            print("💡 Based on the connections above, try adding this to your cred.json:")
            print('   "BING_CONNECTION_NAME": "aibinggrounding"')
            print("\n🔄 Please update your cred.json file and run the code again.")
        else:
            print(f"🔗 Using Bing connection name: {bing_connection_name}")
            print("Proceeding with Bing search agent creation and testing...")
            
            # Create Bing search agent with connection name
            agent = create_bing_search_agent(
                client, 
                config["MODEL_DEPLOYMENT_NAME"], 
                bing_connection_name
            )
            
            # Test agent with queries
            if agent:
                test_bing_search_agent(client, agent)
            
            print("\n✅ Complete Bing search workflow executed successfully!")
        
    else:
        print("❌ BingGroundingTool models are not available")
        print("You may need to install azure-ai-agents package or use a different approach.")

    print("\n✅ Bing search setup completed!")

finally:
    # Clean up resources
    print("✅ Done")

### 6. Cleanup & Best Practices

In [ ]:
# -----------------------------
# Cleanup function
# -----------------------------
def cleanup_bing_resources(client, agent):
    """Clean up created resources"""
    try:
        if agent:
            client.agents.delete_agent(agent.id)
            print("🗑️ Deleted Bing search agent.")
        
        print("✅ Cleanup completed!")
        
    except Exception as e:
        print(f"❌ Error during cleanup: {e}")
cleanup_bing_resources(client, agent)

# Congratulations! 🎉
You've built a **Bing-Grounded Health & Fitness Agent** that can:
1. **Search** the web with Bing.
2. **Answer** health/fitness questions with disclaimers.
3. **Include** references and Bing search query links.

Feel free to expand this approach by combining the BingGroundingTool with other tools (e.g., **FileSearchTool**, **CodeInterpreterTool**) to build a robust advisor.